# 比较pandas和SQL- Comparison with SQL

由于很多pandas用户都对SQL比较熟悉，本节介绍pandas提供的类似SQL的操作。

如果你是pandas新用户，建议你先阅读[10 Minutes to pandas (中文翻译版)](https://github.com/DataMininginAction/pandas-zh/blob/master/10%E5%88%86%E9%92%9F%E6%8E%8C%E6%8F%A1pandas-10%20Minutes%20to%20pandas.ipynb)。

In [1]:
import numpy as np
import pandas as pd

本节大多数是都利用了tips数据集。

In [2]:
url = 'https://raw.github.com/pydata/pandas/master/pandas/tests/data/tips.csv'
tips = pd.read_csv(url)
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## 查询 SELECT

在SQL中，查询使用逗号分隔的列名或者\*表示所有列：

In [ ]:
SELECT total_bill, tip, smoker, time
FROM tips
LIMIT 5;

在pandas中，列查询通过向DataFrame传递列名组成的列表 完成：

In [4]:
tips[['total_bill', 'tip', 'smoker', 'time']].head(5)

,total_bill,tip,smoker,time
0,16.99,1.01,No,Dinner
1,10.34,1.66,No,Dinner
2,21.01,3.50,No,Dinner
3,23.68,3.31,No,Dinner
4,24.59,3.61,No,Dinner


如果仅仅调用DataFrame对象，而不传递具体的列名，作用相当于SELECT \*.

In [5]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## 过滤 WHERE

SQL中过滤条件通过where字句完成：

In [ ]:
SELECT *
FROM tips
WHERE time = 'Dinner'
LIMIT 5;

DataFrame可以使用多种方法进行过滤，最常用的是使用**boolean indexing(布尔索引)**.

In [7]:
tips[tips['time'] == 'Dinner'].head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [8]:
tips[tips.time == 'Dinner'].head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


上面的语句实际上是传递给DataFrame一个值为True/False的Series对象(tips['time'] == 'Dinner')，返回所有的值为True的行。

In [9]:
is_dinner = tips['time'] == 'Dinner'

In [10]:
is_dinner.value_counts()

True     176
False     68
Name: time, dtype: int64

In [11]:
tips[is_dinner].head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


就像SQL中的OR和AND，DataFrame也接受多个条件，使用|(OR)和&(AND)将多个条件结合。

In [ ]:
-- tips of more than $5.00 at Dinner meals
SELECT *
FROM tips
WHERE time ='Dinner' AND tips > 5.00;

In [13]:
#tips of more than $5.00 at Dinner meals
tips[(tips['time']=='Dinner') & (tips['tip'] > 5.00) ]

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
116,29.93,5.07,Male,No,Sun,Dinner,4
155,29.85,5.14,Female,No,Sun,Dinner,5
170,50.81,10.00,Male,Yes,Sat,Dinner,3
172,7.25,5.15,Male,Yes,Sun,Dinner,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2


In [ ]:
-- tips by parties of at least 5 dinners OR bill total was more than $45
SELECT *
FROM tips
WHERE size >= 5 OR total_bill > 45;

In [14]:
# tips by parties of at least 5 diners OR bill total was more than $45
tips[(tips['size'] >= 5) | (tips['total_bill'] > 45)]

,total_bill,tip,sex,smoker,day,time,size
59,48.27,6.73,Male,No,Sat,Dinner,4
125,29.80,4.20,Female,No,Thur,Lunch,6
141,34.30,6.70,Male,No,Thur,Lunch,6
142,41.19,5.00,Male,No,Thur,Lunch,5
143,27.05,5.00,Female,No,Thur,Lunch,6
155,29.85,5.14,Female,No,Sun,Dinner,5
156,48.17,5.00,Male,No,Sun,Dinner,6
170,50.81,10.00,Male,Yes,Sat,Dinner,3
182,45.35,3.50,Male,Yes,Sun,Dinner,3
185,20.69,5.00,Male,No,Sun,Dinner,5


使用notnull() 和isnull()方法检查是否为NULL.

In [17]:
frame = pd.DataFrame({'col1': ['A', 'B', np.NaN, 'C', 'D'],
                     'col2':['F', np.NaN, 'G', 'H', 'I']})

In [18]:
frame

,col1,col2
0,A,F
1,B,NaN
2,NaN,G
3,C,H
4,D,I


如果我们有一个结果和frame相同的表，我们使用下面的SQL语句只能检索col2为NULL的行：

In [ ]:
SELECT *
FROM frame
WHERE col2 IS NULL;

In [20]:
frame[frame['col2'].isnull()]

,col1,col2
1,B,NaN


检索col1不为NULL的使用IS NOT NULL

In [ ]:
SELECT　*
FROM frame
WHERE col1 IS NOT NULL;

In [21]:
frame[frame['col1'].notnull()]

,col1,col2
0,A,F
1,B,NaN
3,C,H
4,D,I


## 分组 GROUP BY

在pandas中，SQL的GROUP BY操作可以用groupby()方法代替。形如其名，groupby()方法作用是将数据集分割为不同组，然后在各个组上运用某些方法，然后将各个组进行聚集。

**group by通常和函数结合使用。**

一个常见的SQL操作是返回每个组的记录数，比如，下面SQL语句:

In [ ]:
SELECT sex, count(*)
FROM tips
GROUP BY sex;

在pandas中，同等的操作:

In [22]:
tips.groupby('sex').size()

sex
Female     87
Male      157
dtype: int64

注意在pandas中我们使用size()而不是count().这是因为count()对每一列数据进行操作！返回每一列not null的记录数！

In [23]:
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,87,87,87,87,87,87
Male,157,157,157,157,157,157


不论是在SQL还是在pandas中，同一个group by操作可以运用多个函数, 比如：

In [ ]:
SELECT day, AVG(tip), COUNT(*)
FROM tips
GROUP BY day;

注意:

**在SQL中，通常SELECT后没有函数操作的列名必须在GROUP BY后出现**。

In [24]:
tips.groupby('day').agg({'tip':np.mean, 'day':np.size})

,tip,day
day,,
Fri,2.734737,19
Sat,2.993103,87
Sun,3.255132,76
Thur,2.771452,62


在SQL和pandas中，使用多个列进行分组当然是可以滴.

In [ ]:
SELECT smoker, day, COUNT(*), AVG(tip)
FROM tips
GROUP BY smoker, day;

In [25]:
tips.groupby(['smoker', 'day']).agg({'tip':[np.size, np.mean]})

tip          
             size      mean
smoker day                 
No     Fri    4.0  2.812500
       Sat   45.0  3.102889
       Sun   57.0  3.167895
       Thur  45.0  2.673778
Yes    Fri   15.0  2.714000
       Sat   42.0  2.875476
       Sun   19.0  3.516842
       Thur  17.0  3.030000

In [27]:
tips.groupby(['smoker', 'day']).agg({'tip':np.size, 'mean':np.mean})

KeyError: 'mean'

## 连接 JOIN